In [2]:
import os
import sys
sys.path.append("..")

In [3]:
from mmengine import Config
from global_config import *

## Modifiable Configuration

In [4]:
FINAL_MODEL_CONFIG_DIR = CONFIG_SEGFORMER_MITB5_DIR_PATH
WORK_DIR = OUT_SEGFORMER_MITB5_DIR_PATH
MODEL_CFG_NAME = "segformer_mitb5"
mmseg_model_cfg_suffix_path = "segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py"
# mmseg_model_cfg_path = Path(MMSEG_MODEL_CONFIG_DIR_PATH)/Path(mmseg_model_cfg_subdir)
mmseg_model_cfg_suffix_path

'segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py'

In [5]:
mmseg_model_cfg_path = os.path.join(MMSEG_MODEL_CONFIG_DIR_PATH, mmseg_model_cfg_suffix_path)
print(mmseg_model_cfg_path)

cfg = Config.fromfile(mmseg_model_cfg_path)
celeba_pipeline_cfg = Config.fromfile(CELEBA_PIPELINE_PATH)
cfg.merge_from_dict(celeba_pipeline_cfg)


/home/featurize/work/AI6126project1/mmsegmentation/configs/segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py


In [6]:
# Modifiable
cfg.data_preprocessor = DATA_PREPROCESSOR
cfg.model.data_preprocessor = DATA_PREPROCESSOR

cfg.norm_cfg = NORM_CFG 

cfg.model.decode_head.norm_cfg = NORM_CFG
cfg.model.decode_head.num_classes = NUM_CLASS


cfg.work_dir = WORK_DIR

cfg.train_dataloader.batch_size = 4
cfg.train_cfg.max_iters = 24000
cfg.train_cfg.val_interval = 500
cfg.default_hooks.logger.interval = 100 
cfg.default_hooks.checkpoint.interval = 500
cfg.default_hooks.checkpoint.max_keep_ckpts = 2
cfg.default_hooks.checkpoint.save_best = 'mIoU'
cfg.default_hooks.checkpoint.rule = "greater"

cfg['randomness'] = dict(seed=0)


In [7]:
cfg.model.decode_head.loss_decode = weighted_cross_lovasz_loss_decode
# cfg.model.auxiliary_head = mitb5_fcn_auxiliary_head

In [8]:
cfg.optimizer = dict(lr=0.0075, momentum=0.9, weight_decay=0.001, type='SGD')
#cfg.optim_wrapper = dict(type='OptimWrapper', optimizer=cfg.optimizer)
cfg.param_scheduler = [
    dict(begin=0, by_epoch=False, end=1000, start_factor=0.1, type='LinearLR'), 
    dict(
        begin=1000,
        by_epoch=False,
        end=cfg.train_cfg.max_iters,   
        eta_min=5e-5,  
        power=0.95,
        type='PolyLR'),
]

In [9]:
#print(mmseg_model_cfg.pretty_text)

In [10]:
from datetime import datetime
ymd_timestamp = datetime.now().strftime("%Y%m%d")
cfg.dump(os.path.join(FINAL_MODEL_CONFIG_DIR, f"celeba_{MODEL_CFG_NAME}_{ymd_timestamp}.py"))
os.path.join(FINAL_MODEL_CONFIG_DIR, f"celeba_{MODEL_CFG_NAME}_{ymd_timestamp}.py")

'/home/featurize/work/AI6126project1/assets/model_configs/segformer_mit-b5/celeba_segformer_mitb5_20250327.py'